# Web-scraping Item Information from Ecommerce Website

#### This project consists of 3 parts:
- Part 1: Download pages of items from ecommerce website.
- Part 2: Loop through the pages downloaded in Part 1, open and parse them into a Python beautifulsoup-object. Identify and select: seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition (e.g., used, new, like new, seller refurbished, ...).
- Part 3: Save item information to MySQL database

#### Assumptions:
- In this project, we search for buy-it-now listings of ipad 4 from ebay and limit the number of items to 100 per page.
- The GET request's variable name corresponding to buy-it-now searches is LH_BIN=1.
- The GET request's variable name corresponding to items per page searches: _ipg=100
- URL: https://www.ebay.com/sch/i.html?_nkw=ipad+4&_in_kw=1&_ex_kw=&_sacat=0&_udlo=&_udhi=&LH_BIN=1&_ftrt=901&_ftrv=1&_sabdlo=&_sabdhi=&_samilow=&_samihi=&_sadis=15&_stpos=94102&_sargn=-1%26saslc%3D1&_salic=1&_sop=12&_dmd=1&_ipg=100&_fosrp=1&_pgn=1


In [4]:
import json
import pandas as pd
import requests
import time
import re
import os
from os import listdir
from bs4 import BeautifulSoup
from tqdm import tqdm
from tqdm._tqdm import trange
import mysql.connector

#### Part 1
#### 1.1 Identify sponsored items and save items'URLs
For the first 10 pages of 100 items/page, save all the URLs of sponsored items' pages to the file "sponsored.txt"  and all the URLs of non-sponsored items' pages to the file "non-sponsored.txt" in the same directory as the code. (One URL per line in each file)

In [2]:
headers = {'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36",
}
url = 'https://www.ebay.com/sch/i.html?_nkw=ipad+4&_in_kw=1&_ex_kw=&_sacat=0&_udlo=&_udhi=&LH_BIN=1&_ftrt=901&_ftrv=1&_sabdlo=&_sabdhi=&_samilow=&_samihi=&_sadis=15&_stpos=94102&_sargn=-1%26saslc%3D1&_salic=1&_sop=12&_dmd=1&_ipg=100&_fosrp=1&_pgn='
sponsorship = ["sponsored", "nonsponsored"]

In [102]:
for i in sponsorship:
    f = open(i+".txt","w", encoding="utf-8") # Create "sponsored.txt" in working directory
    tf = (i == "sponsored")
    for pgn in range(1,11): # For the first 10 pages
        time.sleep(2) # pause between queries
        url_png = url + str(pgn)
        page = requests.get(url_png,headers = headers) # get request
        soup = BeautifulSoup(page.content, 'html.parser') # parse page to beautiful soup object
        items = soup.find_all('li', {"id":re.compile(r".*item.*")}) # locate all items 
        for item in items:                
            a = item.find("a") # locate tag "a"
            if not a:                    
                continue                
            if item.find("div",{"class":"promoted-lv"}):
                sponsor = item.find("div",{"class":"promoted-lv"}).get_text()
            else:
                sponsor = "" # get text "div" tag with "class" = "promoted-lv" if exists  
            if re.search('.*S.*P.*O.*N.*S.*O.*R.*E.*D.*',sponsor) == tf: # if the text contains letters of "sponsor"
                url = re.sub("(.*)\\?.*", r"\1", a['href']) # then get the url in that "a" tag and delete irrelevant variables
                f.write(url_sponsor+"\n") # write the url to txt file
    f.close() # close txt file after the loop

#### 1.2 Download Pages
- Create two folders in the same directory as code and name them "sponsored" and "non-sponsored". 
- Opens the two files in 1.1 and downloads each of the pages (URLs) into the folders "sponsored" and "non-sponsored". Each file should be named as "<item-id>.htm". E.g., "264616053293.htm" for the item with ID "264616053293". 

In [106]:
for i in sponsorship:
    if not os.path.exists(os.path.join(os.getcwd()+'\\'+ i)): # create a folder named "(non)sponsored" in the same directory
        os.makedirs(os.path.join(os.getcwd()+'\\'+ i))
    f_urls = open(i+".txt","r", encoding="utf-8") # open and read "(non)sponsored.txt" 
    urls = str(f_urls.read()) 
    urls = urls.split("\n")
    urls = urls[:-1] # delete the last row which is empty
    for url in urls: # loop through all urls
        id = re.findall(r'[0-9]{12}$',str(url))[0] # item-id is the last 12 numbers in the url
        f  = open(os.path.join(os.getcwd()+'\\'+ i, str(id)+".htm"),"w", encoding="utf-8") # download the page corresponding to the current url
        page = requests.get(url,headers=headers) # get request
        try:
            soup = BeautifulSoup(page.content, 'html.parser') # parse the page to beautiful soup object
            f.write(str(soup)) # save the page 
            f.close()
        except:
            print("Error") 
            time.sleep(2)

#### Part 2 
#### 2.1 Select and Save Item Information
- Loop through the pages downloaded in 1.2 and opens and parses them into a Python beautifulsoup object. 
- Identify and select:
seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition (e.g., used, new, like new, seller refurbished, ...).


In [5]:
# create a dataframe to store information of sponsored items
iitem_info = pd.DataFrame([],columns = ["item_id","sponsored","seller_name", "seller_score", "item_price", "num_items_sold",
                                       "best_offer_available","title", "returns_allowed", "shipping_price", "condition"])

In [5]:
for i in sponsorship:
    filenames = os.listdir(os.getcwd()+'\\'+ i)
    for filename in tqdm(filenames): # for each pages downloaded in c
        f = open(os.path.join(os.getcwd()+'\\'+ i, str(filename)),"r",encoding="utf-8") # open the page
        mysoup = BeautifulSoup(f, 'html.parser') # parse the page to a beautiful soup object
        item_id = re.sub(r"\.htm","",filename)
        if i == "sponsored":
            sponsored = 1
        else:
            sponsored = 0
        seller_name = find_seller_name(mysoup)
        seller_score = find_seller_score(mysoup)
        item_price = find_item_price(mysoup)
        num_items_sold = find_num_items_sold(mysoup)
        best_offer_available = find_best_offer_available(mysoup) 
        title = find_title(mysoup) 
        return_allowed = find_return_allowed(mysoup) 
        shipping_price = find_shipping_price(mysoup)  
        condition = find_condition(mysoup) 
        myadd = pd.DataFrame([[item_id, sponsored, seller_name, seller_score, item_price, num_items_sold, 
                               best_offer_available, title, shipping_price, condition]],
                               columns = ["item_id", "sponsored", "seller_name", "seller_score", "item_price", "num_items_sold", 
                                           "best_offer_available","title", "shipping_price", "condition"])
        item_info = item_info.append(myadd)

100%|██████████| 91/91 [00:41<00:00,  1.69it/s]


In [6]:
def find_seller_name(mysoup):
    try:
        mysi = mysoup.find_all("div",{"class":"si-inner"})[0] # locate tag "div" with "class" = "si-inner" 
        seller_name = mysi.find_all("a",{"aria-label":re.compile(".*Member ID")})[0].get_text().strip()# seller_name is text in tag "a" with unique "aria-label" containg "Member ID"
    except:
        seller_name = ""# in case there is no such tag, assign an empty string 
    return seller_name

In [7]:
def find_seller_score(mysoup):
    try:
        mysi = mysoup.find_all("div",{"class":"si-inner"})[0] # locate tag "div" with "class" = "si-inner" 
        seller_score = mysi.find_all("a",{"title":re.compile(".*feedback score.*")})[0].get_text() # seller_score is text in tag "a" with unqiue "title" containing "feedback"
    except:
        seller_score= ""
    return seller_score

In [8]:
def find_item_price(mysoup):
    try: 
        myprice = mysoup.find_all("div", {"class":"actPanel vi-noborder"})[0] # locate tag "div" with "class" = "actPanel vi-noborder"
        item_price = myprice.find_all("span",{"id":"convbidPrice"})[0].get_text() # Select converted bid price ID if applicable
        item_price = re.sub(r"\(including shipping\)","",item_price) # delete irrelavent strings
    except:
        try: 
            item_price = myprice.find_all("div",{"id":"prcIsumConv"})[0].get_text() # If not, select converted price ID
            item_price = re.sub(r"\(including shipping\)","",item_price) # delete irrelavent strings
            item_price = re.sub(r"Approximately ","",item_price)
        except:
            try: 
                item_price = myprice.find_all("span",{"id":"prcIsum"})[0].get_text() # If not, select normal price ID
            except:
                try: 
                    item_price=myprice.find_all("span",{"id":"mm-saleDscPrc"})[0].get_text() # If not, select sales price ID
                except:
                    try: 
                        item_price=myprice.find_all("span",{"id":"prcIsum_bidPrice"})[0].get_text() # If not, select bid price ID
                    except: 
                        item_price = ""
    return item_price

In [9]:
def find_num_items_sold(mysoup):
    try:
        mysold = mysoup.find_all("span",{"class":re.compile(".*qtyTxt vi-bboxrev-dsplblk vi-qty-fixAlignment feedbackON.*")})[0] # locate the span
        num_items_sold = mysold.find_all("a",{"class":"vi-txt-underline"})[0].get_text() # num_items_sold is usually in underlined format
    except:
        num_items_sold = ""
    return num_items_sold

In [10]:
def find_best_offer_available(mysoup):
    try:
        my_bestoffer = mysoup.find_all("div",{"id":"bstofr"})[0] # uniquely select the div with bestoffer ID which is the childern of the tag interested
        best_offer_available = my_bestoffer.parent.find_all("a",{"id":"boBtn_btn"})[0].get_text().strip() # uniquely get text in parent tag with id attribute
        if best_offer_available == "Make Offer": # if text on button is "Make Offer", then best offer is available
            best_offer_available = 1
    except:
        best_offer_available = 0
    return best_offer_available

In [11]:
def find_title(mysoup):
    try:
        mytitle = mysoup.find_all("h1",{"id":"itemTitle"})[0] # title is in "h1" tag uniquely identified by ID = "itemTitle" 
        title = re.sub(r"(.*</span>)(.*)(</h1>)",r"\2",str(mytitle)) # use regex to delete irrelevant string 
    except:
        title = ""
    return title

In [12]:
def find_return_allowed(mysoup):
    try:
        myreturn = mysoup.find_all("div",{"id":"why2buy"})[0] # uniquely identify "div" tag with ID
        myboxes = myreturn.find_all("span",{"class":"w2b-sgl"}) # "Returns accepted" buttons, if exists, are among the elements under "class" = "w2b-sgl"
        for box in myboxes: # loop through these elements
            if box.get_text().strip() == "Returns accepted": # if matching strings are found, then returns are accepted
                return_allowed = 1
    except:
        return_allowed = 0
    return return_allowed

In [13]:
# shipping_price can be "FREE", "Calculate", "FAST N' FREE", "Free Local Pickup", "Local prickup offered.", or a price number
# select them by corresponding IDs or spans
def find_shipping_price(mysoup):    
    try: 
        shipping_price = mysoup.find_all("span",{"id":"convetedPriceId"})[0].get_text() # in case of price, select converted price first if exists
    except:
        try:
            shipping_price = mysoup.find_all("span",{"id":"fshippingCost"})[0].get_text()
        except:
            try:
                shipping_price = mysoup.find_all("span",{"class":"vi-fnf-ship-txt fnfgreen"})[0].get_text()
            except:
                try:
                    shipping_price = mysoup.find_all("a",{"id":"e3"})[0].get_text()
                except:   
                    try:
                        shipping_price = mysoup.find_all("a",{"id":"e4"})[0].get_text()
                    except: 
                        try:
                            shipping_price = mysoup.find_all("span",{"id":"fShippingSvc"})[0].get_text()
                        except:
                            shipping_price = ""
    return shipping_price

In [14]:
# condition is text in "div" tag uniquely identified by ID = "vi-itm-cond" 
def find_condition(mysoup):
    try: 
        condition = mysoup.find_all("div",{"id":"vi-itm-cond"})[0].get_text().strip()
    except:
        condition = ""
    return condition

#### 2.2 Data Cleaning
- If an item misses ANY of the information in 2.1, insert that missing value as NULL into the table. 
- Convert any price (item price and shipping price) into a "dollar-cent" format (e.g., convert 12.34 into 1234 and 12 into 1200.) Insert the price as INT into the table.

In [ ]:
item_info = item_info.reset_index().drop(["index"],axis = 1)

# Remove string " sold" and "," in num_items_sold and convert it to integer
item_info['num_items_sold']=item_info['num_items_sold'].apply(lambda x: re.sub(r'([0-9]+)( sold)',r'\1', str(x)))
item_info["num_items_sold"] = item_info["num_items_sold"].apply(lambda x: re.sub(r",",r"",str(x)))
item_info["num_items_sold"] = item_info["num_items_sold"].apply(lambda x: int(x) if x!="" else (x))
# Convert seller_score to integer
item_info["seller_score"] = item_info["seller_score"].apply(lambda x: int(x) if x!="" else (x))
# Remove dollar tag and other strings in item_price
item_info["item_price"] = item_info["item_price"].apply(lambda x: re.sub(r"(.*\$)([0-9]+\.*[0-9]*)(.*)",r"\2",str(x)))
# Convert string to float
item_info["item_price"] = item_info["item_price"].apply(lambda x: float(x) if x!="" else (x))
# times 100, now the unit is dollar cent!
item_info["item_price"] = item_info["item_price"].apply(lambda x: str(int(100*x)))

item_info["shipping_price"] = item_info["shipping_price"].apply(lambda x: re.sub(r"\n",r"",str(x)))
# add a new boolean column named shipping_computed which equals to true if shipping_price is "Calculate"
# convert shipping_computed to binary variable
item_info = item_info.assign(shipping_computed = lambda x: item_info["shipping_price"] == "Calculate" )

item_info["shipping_computed"] = item_info["shipping_computed"].apply(lambda x: str(int(x)))
# add a new column named shipping_price_int to store all number shipping price and all free options are considered to be price = 0
item_info["shipping_price_int"] = item_info["shipping_price"]
item_info.loc[item_info['shipping_price'] == "FREE", "shipping_price_int"] = 0
item_info.loc[item_info['shipping_price'] == "FAST 'N FREE", "shipping_price_int"] = 0
item_info.loc[item_info['shipping_price'] == "Free Local Pickup", "shipping_price_int"] = 0
item_info.loc[item_info['shipping_price'] == "				Local pick-up offered.", "shipping_price_int"] = 0
item_info.loc[item_info['shipping_price'] == " See details "] = 0
item_info.loc[item_info['shipping_price'] == "Calculate", "shipping_price_int"] = ""
item_info["shipping_price_int"] = item_info["shipping_price_int"].apply(lambda x: re.sub(r"(.*\$)([0-9]+\.*[0-9]*)(.*)",r"\2",str(x)))
# convert shipping_price_int to float and times 100 now the unit is 100 us dollar cent!
item_info["shipping_price_int"] = item_info["shipping_price_int"].apply(lambda x: float(x) if x!="" else (x))
item_info["shipping_price_int"] = item_info["shipping_price_int"].apply(lambda x: str(int(100*x)) if x!="" else (x))
item_info["condition"] = item_info["condition"].apply(lambda x: "" if x == 0 else (x) )
# convert empty strings to None
item_info = item_info.where(item_info!='', None)

In [33]:
item_info.head()

,item_id,sponsored,seller_name,seller_score,item_price,num_items_sold,best_offer_available,title,returns_allowed,shipping_price,condition,shipping_computed,shipping_price_int
0,114080713473,1,e-cyclepro_ultimate,27659,5599,None,1,"Apple iPad 3,4 A1458 32GB (Wi-Fi Only) 4th Gen...",1,Calculate,Used,1,None
1,123774549197,1,tekreplay,89918,9997,62,0,"Apple iPad 4 | 16GB 32GB 64GB | Wi-Fi 9.7"" - B...",1,FAST 'N FREE,Manufacturer refurbished,0,0
2,133307690525,1,liyuuu013,1656,1049,74,1,OEM 12W USB Power Adapter Wall Charger for App...,1,FAST 'N FREE,New,0,0
3,143404353163,1,wirelessliquidations,2160,11995,363,0,Apple iPad Bundle | 2/3/4/Mini1/2/<wbr/>3/4 | ...,1,FAST 'N FREE,Seller refurbished,0,0
4,143509820763,1,e-cyclepro_ultimate,27659,5583,20,1,"Apple iPad 3,4 A1458 32GB (Wi-Fi Only) Black 4...",1,Calculate,Used,1,None


#### Part 3
#### 3.1 Save Data to MySQL Database
- Connect to MySQL GUI. Create a database and name it "eBay". Save the information of items in Part 2 into a single table named "eBay_items".

In [19]:
mydb = mysql.connector.connect( # connect to mysql
  host="localhost",
  user="root",
  passwd=""
)
mycursor = mydb.cursor() 
mycursor.execute("DROP DATABASE IF EXISTS eBay") # create database eBay if not exists
mycursor.execute("CREATE DATABASE eBay")
mycursor.execute("USE eBay")
mycursor.execute( "CREATE TABLE eBay_items (" # create a table named eBay items
                    "  item_id VARCHAR(255),"
                    "  sponsored INT,"
                    "  seller_name VARCHAR(255),"
                    "  seller_score INT,"
                    "  item_price INT,"
                    "  num_items_sold INT,"
                    "  best_offer_available INT,"
                    "  title VARCHAR(255),"
                    "  returns_allowed INT,"
                    "  shipping_price VARCHAR(255),"
                    "  item_condition VARCHAR(255)," 
                    "  shipping_computed INT,"
                    "  shipping_price_int INT"
                    "  )"
                    
                )
mydb.commit()
mydb.close()

In [20]:
ADD_ITEM_INFO_QUERY = ("INSERT INTO eBay_items "
              "(item_id, sponsored, seller_name, seller_score, item_price, num_items_sold, best_offer_available, title, returns_allowed, shipping_price, item_condition, shipping_computed, shipping_price_int) "
              "VALUES (%(item_id)s, %(sponsored)s, %(seller_name)s, %(seller_score)s, %(item_price)s, %(num_items_sold)s, %(best_offer_available)s, %(title)s, %(returns_allowed)s, %(shipping_price)s, %(item_condition)s, %(shipping_computed)s, %(shipping_price_int)s)")

In [21]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd=""
)
mycursor = mydb.cursor()
mycursor.execute("USE eBay")
for i in item_info.index:  # loop through rows in item_info and insert into table
    ITEM = {
      'item_id': item_info['item_id'][i],
      'sponsored': item_info['sponsored'][i], 
      'seller_name': item_info['seller_name'][i], 
      'seller_score': item_info['seller_score'][i], 
      'item_price': item_info['item_price'][i], 
      'num_items_sold': item_info['num_items_sold'][i], 
      'best_offer_available': item_info['best_offer_available'][i], 
      'title': item_info['title'][i], 
      'returns_allowed': item_info['returns_allowed'][i],
      'shipping_price': item_info['shipping_price'][i],
      'item_condition': item_info['condition'][i],
      'shipping_computed': item_info['shipping_computed'][i],
      'shipping_price_int': item_info['shipping_price_int'][i]
    }
    mycursor.execute(ADD_ITEM_INFO_QUERY, ITEM)
mydb.commit()
mydb.close()

#### 3.2 Summary Stats on Items
- Run summary stats on each item: mean, min, max, and mean for each column, grouped by "sponsor/non-sponsor" and "condition" 
- If it is NOT a numerical/binary categorical column, print to the screen the count of each category level. 
- Ignore NULL values in statistic calculations. 
- Use the stats in and analyze how sponsored and non-sponsored items appear to be different. 

In [30]:
numerical_stats("seller_score")
numerical_stats("item_price")
numerical_stats("num_items_sold")
numerical_stats("best_offer_available")
numerical_stats("returns_allowed")
numerical_stats("shipping_price_int")
categorical_stats("item_condition")

------------------------seller_score---------------------------
   sponsored                 condition         mean   min      max
0          1                      Used   16094.4118    32    32688
1          1  Manufacturer refurbished   87062.9474  1038   382286
2          1                       New   46054.0000  1656   201667
3          1        Seller refurbished  463882.5000  2160  2265747
4          1                  Open box    9952.0000  2172    29577
5          0                       New    8616.5156    20   187058
6          0        Seller refurbished  187299.4828    31  2265914
7          0                      Used   21034.6261    11   311767
8          0  Manufacturer refurbished  102082.2883    52   434822
9          0                  Open box   45757.7031    16   213420
------------------------item_price---------------------------
   sponsored                 condition        mean   min    max
0          1                      Used  10368.7647  3999  24999
1        

#### Findings
- In terms of seller score, min seller scores of non-sponsored items are a lot more lower than sponsored items of the same condition. 
- In terms of item price, on average, sponsored items have lower item price than non-sponsored items of the same condition. Max item prices of non-sponsored items are a lot more higher than sponsored items of the same condition. 
- Non-sponsored items are more likely to have best offers.
- Shipping price of sponsored items are usually free for non-used items. But non-sponsored items have higher shipping price on average.
- In terms of prediction, if best offer is available, the item is more likely to be non-sponsored. If shipping price is not free and even high, the item is more likely to be non-sponsored. 

In [28]:
def numerical_stats(column): # print to screen mean, min, max for each numerical column
    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      passwd="happy100_"
    )
    mycursor = mydb.cursor()
    mycursor.execute("USE eBay")
    # seller_score
    mycursor.execute("select sponsored, item_condition, avg(" + column + "), min(" + column + "), max(" + column + ") \
                      from ebay_items where " + column + " is not null \
                      and item_condition is not null \
                      group by sponsored, item_condition")
    myresult = mycursor.fetchall()
    mydb.commit()
    mydb.close()
    myprint = pd.DataFrame(myresult,columns = ["sponsored","condition","mean","min","max"])
    print ("------------------------" + column + "---------------------------")
    print (myprint)

In [29]:
def categorical_stats(column): # print to screen the count of each category level for each categorical column
    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      passwd="happy100_"
    )
    mycursor = mydb.cursor()
    mycursor.execute("USE eBay")
    # seller_score
    mycursor.execute("select sponsored, item_condition, count(" + column + ")\
                      from ebay_items where " + column + " is not null \
                     group by sponsored, item_condition")
    myresult = mycursor.fetchall()
    mydb.commit()
    mydb.close()
    myprint = pd.DataFrame(myresult,columns = ["sponsored","condition","count"])
    print ("------------------------" + column + "---------------------------")
    print (myprint)